In [ ]:
import os
from google.colab import drive

# ============================================================
# ⚙️ 전역 환경 설정 (모든 설정을 여기서 관리)
# ============================================================

# --- Google Drive 마운트 (이미 마운트된 경우 건너뜀) ---
if not os.path.exists('/content/drive/MyDrive'):
    print("📂 Google Drive 마운트 중...")
    drive.mount('/content/drive')
    print("✅ Google Drive 마운트 완료")
else:
    print("✅ Google Drive 이미 마운트됨")

# --- 경로 설정 ---
DRIVE_ROOT = '/content/drive/MyDrive'
SGF_ROOT = os.path.join(DRIVE_ROOT, 'baduk/Pro')
CHECKPOINT_DIR = os.path.join(DRIVE_ROOT, 'checkpoints')
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# SGF 파일을 검색할 모든 서브폴더 자동 수집 (1p, 2p, ..., 9p 등)
SGF_SEARCH_ROOTS = []
if os.path.exists(SGF_ROOT):
    # Pro 폴더 내의 모든 디렉토리 검색
    for folder in sorted(os.listdir(SGF_ROOT)):
        folder_path = os.path.join(SGF_ROOT, folder)
        if os.path.isdir(folder_path):
            SGF_SEARCH_ROOTS.append(folder_path)
    print(f"✅ 발견된 데이터 폴더: {len(SGF_SEARCH_ROOTS)}개")
    for root in SGF_SEARCH_ROOTS:
        print(f"   * {os.path.basename(root)}")
else:
    print(f"⚠️ SGF 경로를 찾을 수 없습니다: {SGF_ROOT}")
    SGF_SEARCH_ROOTS = []

# --- 바둑판 및 모델 설정 ---
BOARD_SIZE = 19
NUM_ACTIONS = BOARD_SIZE * BOARD_SIZE  # 361
NUM_CHANNELS = 3  # 흑돌, 백돌, 차례

# --- 데이터 처리 설정 ---
BATCH_GAMES_SIZE = 500  # 한 번에 파싱할 게임 개수 (10GB 메모리: 500)
                        # 메모리 부족 시 200으로 감소, 속도 우선 시 1000으로 증가

# --- 모델 학습 설정 ---
BATCH_SIZE = 256  # 모델 학습 배치 크기 (10GB 메모리: 256)
EPOCHS = 5  # 총 에포크 수
LEARNING_RATE = 0.001  # 학습률

# --- 모니터링 설정 ---
VERBOSE_MODE = True  # True: 상세 출력, False: 간단한 출력
PROGRESS_INTERVAL = 10  # 진행 상태 업데이트 간격 (%)

print("=" * 60)
print("⚙️  전역 환경 설정 로드 완료")
print("=" * 60)
print(f"📂 기본 경로: {SGF_ROOT}")
print(f"💾 체크포인트: {CHECKPOINT_DIR}")
print(f"🎯 바둑판: {BOARD_SIZE}x{BOARD_SIZE}")
print(f"📦 배치 크기: {BATCH_GAMES_SIZE}게임 (데이터), {BATCH_SIZE}샘플 (모델)")
print(f"📚 학습 설정: {EPOCHS}에포크, 학습률={LEARNING_RATE}")
print("=" * 60)

In [ ]:
# --- 4. SGF 파일 검색 및 개수 입력 ---
print(f"⭐ SGF 파일 검색 시작")
print(f"📂 검색 폴더: {len(SGF_SEARCH_ROOTS)}개\n")

sgf_file_paths = find_sgf_files(SGF_SEARCH_ROOTS)
total_found = len(sgf_file_paths)
print(f"✅ 총 {total_found}개의 .sgf 파일을 찾았습니다.\n")

NameError: name 'SGF_SEARCH_ROOT' is not defined

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import gc
import pickle

# ============================================================
# 🎓 배치 기반 모델 학습
# ============================================================

# 체크포인트 파일 경로
TRAINING_CHECKPOINT_FILE = os.path.join(CHECKPOINT_DIR, 'model_training.pkl')

# --- 1. Keras 모델 정의 ---
def build_policy_model(input_shape, num_actions):
    """정책망(Policy Network) 구축"""
    inputs = keras.Input(shape=input_shape)

    # 초기 컨볼루션 레이어
    x = layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)

    # 레지듀얼 블록
    def residual_block(y, filters):
        shortcut = y
        y = layers.Conv2D(filters, kernel_size=3, padding='same', activation='relu')(y)
        y = layers.BatchNormalization()(y)
        y = layers.Conv2D(filters, kernel_size=3, padding='same', activation='relu')(y)
        y = layers.BatchNormalization()(y)
        y = layers.Add()([shortcut, y])
        return y

    for _ in range(3):
        x = residual_block(x, filters=128)

    # 정책 헤드
    x = layers.Conv2D(filters=2, kernel_size=1, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(num_actions, activation='softmax', name='policy_output')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name='go_policy_network')
    return model

# 모델 생성 및 컴파일
input_shape = (BOARD_SIZE, BOARD_SIZE, NUM_CHANNELS)
policy_model = build_policy_model(input_shape, NUM_ACTIONS)

policy_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("## 🧱 정책망 모델 요약")
policy_model.summary()
print("\n" + "=" * 60)

# --- 3. 배치 파일 수집 및 학습 ---
def get_available_batches():
    """저장된 배치 파일 목록 반환"""
    batch_files = []
    if os.path.exists(CHECKPOINT_DIR):
        for f in os.listdir(CHECKPOINT_DIR):
            if f.startswith('batch_') and f.endswith('.npz'):
                batch_files.append(os.path.join(CHECKPOINT_DIR, f))
    return sorted(batch_files)

def load_training_checkpoint():
    """학습 체크포인트 로드"""
    if os.path.exists(TRAINING_CHECKPOINT_FILE):
        with open(TRAINING_CHECKPOINT_FILE, 'rb') as f:
            return pickle.load(f)
    return {'batches_processed': 0, 'total_epochs_done': 0, 'total_samples': 0}

def save_training_checkpoint(checkpoint):
    """학습 체크포인트 저장"""
    with open(TRAINING_CHECKPOINT_FILE, 'wb') as f:
        pickle.dump(checkpoint, f)

print("\n🚀 배치 기반 모델 학습 시작")
print("=" * 60)

available_batches = get_available_batches()
total_batches = len(available_batches)

if total_batches == 0:
    print("❌ 배치 파일을 찾을 수 없습니다. 먼저 Cell 2를 실행하세요.")
else:
    training_checkpoint = load_training_checkpoint()
    total_epochs_done = training_checkpoint['total_epochs_done']
    total_samples_trained = training_checkpoint['total_samples']

    print(f"📊 사용 가능한 배치: {total_batches}개")
    print(f"📈 이전 진행: {total_epochs_done}회 학습, {total_samples_trained:,}개 샘플\n")
    
    # 에포크별 학습
    for epoch in range(EPOCHS):
        print(f"\n{'='*60}")
        print(f"📚 Epoch {epoch + 1}/{EPOCHS}")
        print(f"{'='*60}")

        epoch_loss = 0
        epoch_accuracy = 0
        batch_count = 0

        # 배치별 학습
        for batch_idx, batch_file in enumerate(available_batches):
            # 배치 데이터 로드
            X_batch, Y_batch = load_batch_data(batch_file)
            
            print(f"  > 배치 {batch_idx + 1}/{total_batches} 학습 중... ({X_batch.shape[0]}개 샘플)")

            # 배치 학습
            history = policy_model.fit(
                X_batch, Y_batch,
                batch_size=BATCH_SIZE,
                epochs=1,
                verbose=0,
                shuffle=True,
                validation_split=0.1
            )

            epoch_loss += history.history['loss'][0]
            epoch_accuracy += history.history['accuracy'][0]
            batch_count += 1
            total_samples_trained += X_batch.shape[0]

            # 메모리 정리
            del X_batch, Y_batch
            gc.collect()

        # 에포크 결과 출력
        avg_loss = epoch_loss / batch_count
        avg_accuracy = epoch_accuracy / batch_count
        print(f"\n✅ Epoch {epoch + 1} 완료")
        print(f"  * 평균 손실: {avg_loss:.4f}")
        print(f"  * 평균 정확도: {avg_accuracy:.4f}")
        print(f"  * 누적 학습 샘플: {total_samples_trained:,}개")

        # 체크포인트 저장
        training_checkpoint = {
            'batches_processed': total_batches,
            'total_epochs_done': total_epochs_done + epoch + 1,
            'total_samples': total_samples_trained
        }
        save_training_checkpoint(training_checkpoint)

    print("\n" + "=" * 60)
    print("✅ 배치 기반 학습 완료!")
    print(f"* 총 처리 배치: {total_batches}개")
    print(f"* 총 학습 샘플: {total_samples_trained:,}개")
    print("=" * 60)

    # 모델 저장
    model_save_path = os.path.join(CHECKPOINT_DIR, "go_policy_network_checkpoint.h5")
    try:
        policy_model.save(model_save_path)
        print(f"\n💾 모델 저장: {model_save_path}")
    except Exception as e:
        print(f"\n⚠️ 모델 저장 실패: {e}")

## 🧱 정책망 모델 요약


Model: "go_policy_network"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 19, 19, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 19, 19,    │      3,584 │ input_layer[0][0] │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 19, 19,    │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 19, 19,    │    147,584 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        512 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 19, 19,    │    147,584 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 19, 19,    │          0 │ batch_normalizat… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 19, 19,    │    147,584 │ add[0][0]         │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 19, 19,    │    147,584 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        512 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 19, 19,    │          0 │ add[0][0],        │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 19, 19,    │    147,584 │ add_1[0][0]       │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        512 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 19, 19,    │    147,584 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        512 │ conv2d_6[0][0]  

 Total params: 1,153,941 (4.40 MB)

 Trainable params: 1,152,145 (4.40 MB)

 Non-trainable params: 1,796 (7.02 KB)

--------------------------------------------------

🚀 모델 지도학습 시작 (총 10 Epochs)...


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# ============================================================
# 🎮 학습된 모델 로드 및 추론
# ============================================================

# 모델 로드
model_path = os.path.join(CHECKPOINT_DIR, "go_policy_network_checkpoint.h5")
if os.path.exists(model_path):
    policy_model = keras.models.load_model(model_path)
    print(f"✅ 모델 로드: {model_path}")
else:
    print(f"⚠️ 저장된 모델을 찾을 수 없습니다: {model_path}")
    print("이전 셀에서 모델을 학습해주세요.")

class GoBoard:
    def __init__(self, size=BOARD_SIZE):
        self.size = size
        # 0: 빈 칸, 1: 흑돌 (Black), 2: 백돌 (White)
        self.board = np.zeros((size, size), dtype=np.int32)
        self.current_player = 1  # 1: 흑돌(Black), 2: 백돌(White)
        self.player_map = {1: 'B', 2: 'W'}

    def get_state(self):
        """
        Policy Network의 입력 형태 (19, 19, 3)으로 변환합니다.
        채널 0: 흑돌 위치 (19x19)
        채널 1: 백돌 위치 (19x19)
        채널 2: 현재 플레이어 (19x19) -> 현재 턴이 흑돌이면 모두 1, 백돌이면 모두 0
        """
        # (19, 19)
        black_stones = (self.board == 1).astype(np.float32)
        white_stones = (self.board == 2).astype(np.float32)

        # 현재 플레이어 정보 (흑=1, 백=0)
        player_color = np.full((self.size, self.size),
                               1.0 if self.current_player == 1 else 0.0,
                               dtype=np.float32)

        # (19, 19, 3) 형태로 쌓기
        state = np.stack([black_stones, white_stones, player_color], axis=-1)

        # 모델은 배치 입력을 기대하므로, 배치 차원 추가 (1, 19, 19, 3)
        return np.expand_dims(state, axis=0)

    def display(self):
        """바둑판을 텍스트로 출력합니다."""
        print("-" * (self.size * 2 + 1))
        # 헤더 (A-T)
        header = "  " + " ".join([chr(ord('A') + i) for i in range(self.size)])
        print(header)

        for r in range(self.size):
            row_str = f"{r+1:2d}"
            for c in range(self.size):
                if self.board[r, c] == 1:
                    row_str += ' X'  # 흑돌
                elif self.board[r, c] == 2:
                    row_str += ' O'  # 백돌
                else:
                    row_str += ' .'  # 빈 칸
            print(row_str)
        print("-" * (self.size * 2 + 1))
        print(f"현재 차례: {self.player_map[self.current_player]}")

    def is_valid_move(self, r, c):
        """간단한 착수 가능 여부 확인 (경계 및 빈 칸만 확인)"""
        if 0 <= r < self.size and 0 <= c < self.size and self.board[r, c] == 0:
            return True
        return False

    def make_move(self, r, c):
        """돌을 놓고 턴을 바꿉니다."""
        if not self.is_valid_move(r, c):
            return False

        self.board[r, c] = self.current_player
        self.current_player = 3 - self.current_player  # 1 -> 2, 2 -> 1로 변경
        return True

    def get_ai_move(self, policy_model, top_k=5):
        """
        Policy Network를 사용하여 AI의 최적 수를 결정합니다.
        상위 top_k개의 후보 중에서 가능한 수를 선택합니다.
        """
        state = self.get_state()
        predictions = policy_model.predict(state, verbose=0)[0]  # (361,)
        
        # 상위 top_k개의 인덱스 추출
        top_indices = np.argsort(predictions)[-top_k:][::-1]
        
        for idx in top_indices:
            r = idx // self.size
            c = idx % self.size
            if self.is_valid_move(r, c):
                return r, c, predictions[idx]
        
        # 모든 top_k가 불가능하면 임의의 빈 칸에 둡니다.
        empty_positions = np.where(self.board == 0)
        if len(empty_positions[0]) > 0:
            rand_idx = np.random.randint(len(empty_positions[0]))
            return empty_positions[0][rand_idx], empty_positions[1][rand_idx], 0.0
        
        return None, None, None

print("=" * 50)
print("✅ GoBoard 클래스 준비 완료")
print("=" * 50)

# 🎯 통합 학습 플로우 가이드

## ⚙️ 환경 설정 (Cell 1)

**모든 설정을 한 곳에서 관리합니다:**

```python
# 경로 설정
DRIVE_ROOT = '/content/drive/MyDrive'
SGF_SEARCH_ROOT = os.path.join(DRIVE_ROOT, 'baduk/Pro')

# 바둑판 설정
BOARD_SIZE = 19
NUM_ACTIONS = 361  # 자동 계산

# 데이터 처리 설정
BATCH_GAMES_SIZE = 500  # 한 번에 파싱할 게임 개수
                        # 메모리 부족 시: 200, 속도 우선 시: 1000

# 모델 학습 설정
BATCH_SIZE = 256      # 모델 학습 배치 크기
EPOCHS = 5            # 총 에포크 수
LEARNING_RATE = 0.001 # 학습률

# 모니터링 설정
VERBOSE_MODE = True   # 상세 출력 여부
```

## 📊 실행 순서

1. **Cell 1** ⚙️ **전역 환경 설정**
   - 모든 설정을 한 곳에서 관리
   - Google Drive 마운트
   
2. **Cell 2** 🚀 **SGF 파싱 + 배치 처리 (통합)**
   - 학습할 파일 개수 입력
   - 500게임씩 배치로 처리
   - 자동으로 파싱 → 데이터 증강 → 배치 저장
   - 체크포인트로 중단/재개 가능
   
3. **Cell 3** 🎓 **배치 기반 모델 학습**
   - 저장된 배치 파일을 순차 로드
   - 5 에포크 학습 (배치 크기 256)
   - 에포크 완료 후 자동 체크포인트 저장
   
4. **Cell 4** 🎮 **학습된 모델 로드 및 추론**
   - 모델 자동 로드
   - GoBoard 클래스로 추론
   
5. **Cell 5** 📊 **메모리/진행 상황 모니터링**
   - 메모리 사용량 확인
   - 진행 상황 통계

## 🚀 핵심 특징

### ⚙️ 중앙 집중식 설정 관리
- ✅ 모든 하이퍼파라미터를 Cell 1에서 관리
- ✅ 다른 셀들은 전역 변수 사용
- ✅ 설정 변경 시 Cell 1만 수정

### 📦 통합 처리
- ✅ 파싱을 한 번만 수행 (중복 제거)
- ✅ 파싱 → 데이터 증강 → 배치 저장을 하나의 셀에서 처리
- ✅ 파일 개수 입력도 같은 셀에서 처리

### 💾 메모리 최적화
- ✅ 500게임씩 처리 (10GB 메모리 활용)
- ✅ 배치당 약 30초만 소요
- ✅ 1000개 파일: 약 12분

## 💡 사용 방법

### 첫 실행
```
1. Cell 1 실행 (1분) - 설정 로드
2. Cell 2 실행 → "1000" 입력 → 12분 대기
3. Cell 3 실행 → 10분 대기
4. Cell 4 실행 → 모델 추론
```

### 설정 변경
```python
# Cell 1의 다음 값을 수정:
BATCH_GAMES_SIZE = 200  # 메모리 절약
EPOCHS = 10             # 더 많은 학습
BATCH_SIZE = 128        # 작은 배치
```

### 추가 학습
```
1. Cell 1 실행 (설정 확인)
2. Cell 2 실행 → "2000" 입력 (1000개 추가)
   → 1000개는 건너뜀, 1000개만 처리
3. Cell 3 실행 → 자동으로 모든 배치 학습
```

### 모델만 재학습
```
1. Cell 1 실행
2. Cell 3 실행 → 기존 배치로 재학습
```


In [ ]:
import psutil

# ============================================================
# 📊 메모리 및 진행 상황 모니터링
# ============================================================

def print_memory_stats():
    """현재 메모리 사용 상황 출력"""
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    
    rss_mb = mem_info.rss / (1024 ** 2)
    total_mem = psutil.virtual_memory().total / (1024 ** 3)
    available_mem = psutil.virtual_memory().available / (1024 ** 3)
    used_mem = psutil.virtual_memory().used / (1024 ** 3)
    
    print("=" * 60)
    print("💾 메모리 사용 통계")
    print("=" * 60)
    print(f"🔴 현재 프로세스 메모리: {rss_mb:.2f} MB")
    print(f"💻 시스템 전체 메모리")
    print(f"   * 총: {total_mem:.2f} GB | 사용: {used_mem:.2f} GB")
    print(f"   * 가용: {available_mem:.2f} GB | 사용률: {psutil.virtual_memory().percent}%")
    print("=" * 60)

def print_checkpoint_stats():
    """체크포인트 통계 출력"""
    if not os.path.exists(CHECKPOINT_DIR):
        print("❌ 체크포인트 디렉토리를 찾을 수 없습니다.")
        return
    
    # 배치 파일 통계
    batch_files = [f for f in os.listdir(CHECKPOINT_DIR) 
                   if f.startswith('batch_') and f.endswith('.npz')]
    total_batch_size = sum(os.path.getsize(os.path.join(CHECKPOINT_DIR, f)) 
                          for f in batch_files) / (1024 ** 2) if batch_files else 0
    
    print("\n" + "=" * 60)
    print("📊 체크포인트 통계")
    print("=" * 60)
    print(f"📦 배치 파일: {len(batch_files)}개 ({total_batch_size:.2f} MB)")
    
    # 데이터 처리 체크포인트
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'rb') as f:
            checkpoint = pickle.load(f)
        print(f"📂 데이터 처리")
        print(f"   * 처리 파일: {checkpoint['games_processed']}개")
        print(f"   * 생성 배치: {checkpoint['batch_count']}개")
    
    # 모델 학습 체크포인트
    if os.path.exists(TRAINING_CHECKPOINT_FILE):
        with open(TRAINING_CHECKPOINT_FILE, 'rb') as f:
            checkpoint = pickle.load(f)
        print(f"📈 모델 학습")
        print(f"   * 완료 에포크: {checkpoint['total_epochs_done']}회")
        print(f"   * 학습 샘플: {checkpoint['total_samples']:,}개")
    
    print("=" * 60)

def reset_all_checkpoints():
    """모든 체크포인트 초기화 (처음부터 시작할 때만 사용)"""
    import shutil
    if os.path.exists(CHECKPOINT_DIR):
        shutil.rmtree(CHECKPOINT_DIR)
        os.makedirs(CHECKPOINT_DIR)
        print("✅ 모든 체크포인트를 초기화했습니다.")
    else:
        print("⚠️ 체크포인트 디렉토리가 없습니다.")

# --- 통계 출력 ---
print("\n")
print_memory_stats()
print_checkpoint_stats()

print("\n" + "=" * 60)
print("🎯 다음 단계")
print("=" * 60)
print("1️⃣ Cell 2 실행: SGF 파싱 및 배치 처리")
print("2️⃣ Cell 3 실행: 모델 학습")
print("3️⃣ Cell 4 실행: 모델 추론")
print("\n💡 처음부터 시작하려면: reset_all_checkpoints()")
print("=" * 60)